In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

C:\Users\artis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [10]:
# folders = glob.glob('/content/drive/MyDrive/ai_project/data_sample200/train/*')
all_img_list = glob.glob('C:\dob\\train_images/*/*')
# root = '/content/drive/MyDrive/ai_project/data_sample200/train'
# test_folder = '/content/drive/MyDrive/ai_project/test'
print(all_img_list)

['C:\\dob\\train_images\\가구수정\\0.png', 'C:\\dob\\train_images\\가구수정\\1.png', 'C:\\dob\\train_images\\가구수정\\10.png', 'C:\\dob\\train_images\\가구수정\\11.png', 'C:\\dob\\train_images\\가구수정\\2.png', 'C:\\dob\\train_images\\가구수정\\3.png', 'C:\\dob\\train_images\\가구수정\\4.png', 'C:\\dob\\train_images\\가구수정\\5.png', 'C:\\dob\\train_images\\가구수정\\6.png', 'C:\\dob\\train_images\\가구수정\\7.png', 'C:\\dob\\train_images\\가구수정\\8.png', 'C:\\dob\\train_images\\가구수정\\9.png', 'C:\\dob\\train_images\\가구수정\\br0_0.png', 'C:\\dob\\train_images\\가구수정\\br0_1.png', 'C:\\dob\\train_images\\가구수정\\br0_10.png', 'C:\\dob\\train_images\\가구수정\\br0_11.png', 'C:\\dob\\train_images\\가구수정\\br0_2.png', 'C:\\dob\\train_images\\가구수정\\br0_3.png', 'C:\\dob\\train_images\\가구수정\\br0_4.png', 'C:\\dob\\train_images\\가구수정\\br0_5.png', 'C:\\dob\\train_images\\가구수정\\br0_6.png', 'C:\\dob\\train_images\\가구수정\\br0_7.png', 'C:\\dob\\train_images\\가구수정\\br0_8.png', 'C:\\dob\\train_images\\가구수정\\br0_9.png', 'C:\\dob\\train_images\\가구수정\\br1_0

In [3]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'LEARNING_RATE':0.01,
    'BATCH_SIZE':48,
    'EARLY_STOPPING': 10,
    'SEED': 10,
}

In [4]:
# fix seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [12]:
# data labeling and grouping

df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[3])
le = preprocessing.LabelEncoder()
df['label']=le.fit_transform(df['label'])
print("class:", le.classes_, "\ntotal img:", len(df['img_path']))
print(df['label'].value_counts())

class: ['가구수정' '걸레받이수정' '곰팡이' '꼬임' '녹오염' '들뜸' '면불량' '몰딩수정' '반점' '석고수정' '오염' '오타공'
 '울음' '이음부불량' '창틀,문틀수정' '터짐' '틈새과다' '피스' '훼손'] 
total img: 11307
label
18    1405
13    1239
9     1197
5     1134
3      630
17     603
10     595
14     567
15     486
12     462
2      435
11     426
7      390
8      378
1      307
6      297
4      294
0      252
16     210
Name: count, dtype: int64


In [22]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        img_array = np.fromfile(img_path, np.uint8)
        image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        # image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [23]:
res_transform = A.Compose([
                          A.Resize(224,224),
                          A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                          ToTensorV2()
                          ])

In [24]:
train, val, _,_  = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])
print("train:", len(train), "validation:", len(val))

train: 7914 validation: 3393


In [25]:
# dataset

batch = 48
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, res_transform)
train_loader = DataLoader(train_dataset, batch_size = batch, shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, res_transform)
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=False, num_workers=0)

In [26]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [31]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    es_count = 0
    best_epoch = 0
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.long().to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        es_count += 1
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            print(f"Epoch [{epoch}] new best score")
            es_count = 0
            best_score = _val_score
            best_model = model
            best_epoch = epoch
        if es_count == CFG['EARLY_STOPPING']:
            print(f"EARLY STOPPING COUNT: {CFG['EARLY_STOPPING']} BEST EPOCH: {best_epoch}")
            break
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}], es count: {es_count}')
        print("------------------------------------------------------------------------------------------------------------------------------")
    
    return best_model

In [32]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            img=imgs
            imgs = imgs.float().to(device)
            labels = labels.long().to(device)
            
            pred = model(imgs)
           
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [33]:
# model load

model = BaseModel()

In [34]:
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 6.2500e-05)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 71/71 [08:22<00:00,  7.08s/it]


Epoch [1] new best score
Epoch [1], Train Loss : [0.82463] Val Loss : [0.25206] Val Weighted F1 Score : [0.92527], es count: 0
------------------------------------------------------------------------------------------------------------------------------


 38%|███▊      | 62/165 [22:32<37:26, 21.81s/it]


KeyboardInterrupt: 

In [ ]:
res = pd.read_csv('/content/drive/MyDrive/ai_project/test.csv')
res_dataset = CustomDataset('/content/drive/MyDrive/ai_project/'+res['img_path'].values, None, res_transform)
res_loader = DataLoader(res_dataset, batch_size=48, shuffle=False, num_workers=0)

In [ ]:
def label_decoder(label):
    new_label =[
        '가구수정' if l==0 else
        '걸레받이수정' if l==1 else
        '곰팡이' if l==2 else
        '꼬임' if l==3 else
        '녹오염' if l==4 else
        '들뜸' if l==5 else
        '면불량' if l==6 else
        '몰딩수정' if l==7 else
        '반점' if l==8 else
        '석고수정' if l==9 else
        '오염' if l==10 else
        '오타공' if l==11 else
        '울음' if l==12 else
        '이음부불량' if l==13 else
        '창틀,문틀수정' if l==14 else
        '터짐' if l==15 else
        '틈새과다' if l==16 else
        '피스' if l==17 else
        '훼손' if l==18 else
        'NaN' for l in label
    ]
    return np.array(new_label)

In [ ]:
def inference(model, res_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(res_loader)):
            imgs = imgs.float().to(device)
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()

    pred_kor=label_decoder(preds)
    
    # preds = le.inverse_transform(preds)
    return pred_kor

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/ai_project/test.csv')
test_dataset = CustomDataset('/content/drive/MyDrive/ai_project/'+test['img_path'].values, None, res_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
preds = inference(infer_model, test_loader, device)
preds = [str(x) for x in preds]

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/ai_project/sample_submission.csv',encoding='utf-8-sig')
submit['label'] = preds
submit.to_csv('/content/drive/MyDrive/ai_project/baseline_submit_densenet.csv', index=False, encoding='utf-8-sig')

In [ ]:
# time

import datetime as dt
x = dt.datetime.now()
time = f'{x.month}_{x.day}_{x.hour + 9}:{x.minute}:{x.second}'

save weights

In [ ]:
savePath = f'/content/drive/MyDrive/ai_project/samll_{CFG["EPOCHS"]}_{time}.pt'
torch.save(model.state_dict(), savePath)

load weights

In [ ]:
model = BaseModel()
model.load_state_dict(torch.load(savePath))
model.eval()